### **Step 1: Load and Clean Data**

### **Genders Are male & female**

### **Match Types Are: ODI, T20, Test, MDM, ODM**



In [2]:
import pandas as pd

# Input match type and gender
match_type = input("Enter Match Type:")
gender = input("Enter Gender:")

print("Gender is: ", gender.capitalize())
print("Match Type is: ", match_type.capitalize())

# Load the match summary data
match_summary = pd.read_csv(rf"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\{gender}\{match_type}_{gender}_match_summary_cleaned.csv")
#match_summary.drop(columns=["Unnamed: 0"], inplace=True)

# Select relevant columns
summary = match_summary[['match_id', 'season', 'match_date', 'event_name', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'ground']].copy()
summary

Gender is:  Male
Match Type is:  T20


,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground
0,1449636,2025,2025-01-11,SA20,Sunrisers Eastern Cape,Paarl Royals,Sunrisers Eastern Cape,bat,Paarl Royals,"Boland Bank Park, Paarl"
1,1443087,2025,2025-01-11,Big Bash League,Adelaide Strikers,Brisbane Heat,Brisbane Heat,field,Adelaide Strikers,"Adelaide Oval, Adelaide"
2,1443086,2025,2025-01-11,Big Bash League,Sydney Sixers,Perth Scorchers,Perth Scorchers,field,Sydney Sixers,"Sydney Cricket Ground, Sydney"
3,1459552,2025,2025-01-10,Bangladesh Premier League,Dhaka Capital,Sylhet Strikers,Dhaka Capital,bat,Sylhet Strikers,"Sylhet International Cricket Stadium, Academy ..."
4,1459551,2025,2025-01-10,Bangladesh Premier League,Durbar Rajshahi,Khulna Tigers,Khulna Tigers,field,Durbar Rajshahi,"Sylhet International Cricket Stadium, Academy ..."
...,...,...,...,...,...,...,...,...,...,...
8729,237242,2006,2006-02-16,NaN,New Zealand,West Indies,New Zealand,field,tie,"Eden Park, Auckland"
8730,226374,2006,2006-01-09,NaN,Australia,South Africa,Australia,bat,Australia,"Brisbane Cricket Ground, Woolloongabba, Brisbane"
8731,222678,2005,2005-10-21,NaN,South Africa,New Zealand,New Zealand,field,New Zealand,"New Wanderers Stadium, Johannesburg"
8732,211028,2005,2005-06-13,NaN,England,Australia,England,bat,England,"The Rose Bowl, Southampton"


### **Step 2: Unique Event Names**

In [3]:
# Get unique events and count
unique_events = summary["event_name"].unique()
num_events = summary["event_name"].nunique()

# Save unique events to CSV
events_df = pd.DataFrame({"Unique Events": unique_events})
events_df.to_csv(rf"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\{gender}\{gender}_unique_events_summary.csv", index=False)

print("No of Events:", num_events)

No of Events: 362


### **Step 3: Swap Teams to Maintain Consistent Order**

In [4]:
# Create a column for sorted team tuples
summary["team_tuple"] = summary.apply(lambda row: tuple(sorted([row["team1"], row["team2"]])), axis=1)

# Identify tuples that appear in reverse order and swap team names where necessary
reverse_tuples = summary["team_tuple"].value_counts()[summary["team_tuple"].value_counts() > 1].index
for team_tuple in reverse_tuples:
    team1, team2 = team_tuple
    summary.loc[(summary["team1"] == team1) & (summary["team2"] == team2), ["team1", "team2"]] = team2, team1

# Drop the temporary column and filter the dataset
summary.drop(columns=["team_tuple"], inplace=True)
summary = summary[(summary["winner"] == summary["team1"]) | (summary["winner"] == summary["team2"])].sort_values(by="match_id", ascending=False).reset_index(drop=True)
summary.reset_index().sort_values(by=["match_date", "match_id"], ascending=False, inplace=True)
summary[:10]

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground
0,1463667,2024,2024-12-21,Gulf T20I Championship,United Arab Emirates,Kuwait,United Arab Emirates,bat,United Arab Emirates,"Dubai International Cricket Stadium, Dubai"
1,1463666,2024,2024-12-20,Gulf T20I Championship,Oman,Kuwait,Oman,bat,Kuwait,"ICC Global Cricket Academy, Dubai"
2,1463665,2024,2024-12-20,Gulf T20I Championship,Qatar,Bahrain,Qatar,field,Qatar,"ICC Global Cricket Academy, Dubai"
3,1463664,2024,2024-12-19,Gulf T20I Championship,United Arab Emirates,Saudi Arabia,United Arab Emirates,field,Saudi Arabia,"ICC Global Cricket Academy, Dubai"
4,1463663,2024,2024-12-18,Gulf T20I Championship,United Arab Emirates,Qatar,United Arab Emirates,bat,United Arab Emirates,"ICC Global Cricket Academy, Dubai"
5,1463662,2024,2024-12-18,Gulf T20I Championship,Saudi Arabia,Oman,Saudi Arabia,field,Saudi Arabia,"ICC Global Cricket Academy, Dubai"
6,1463661,2024,2024-12-17,Gulf T20I Championship,Oman,Bahrain,Bahrain,field,Oman,"ICC Global Cricket Academy, Dubai"
7,1463660,2024,2024-12-17,Gulf T20I Championship,Qatar,Kuwait,Kuwait,field,Kuwait,"ICC Global Cricket Academy, Dubai"
8,1463659,2024,2024-12-16,Gulf T20I Championship,United Arab Emirates,Kuwait,United Arab Emirates,bat,United Arab Emirates,"ICC Global Cricket Academy, Dubai"
9,1463658,2024,2024-12-16,Gulf T20I Championship,Saudi Arabia,Bahrain,Bahrain,field,Bahrain,"ICC Global Cricket Academy, Dubai"


### **Step 4: Head-to-Head Winning Count**

In [5]:
# Calculate total matches played and head-to-head wins for team1
summary['h2h'] = summary.groupby(['team1', 'team2'])['match_id'].transform('count')
h2h_wins = summary.groupby(['team1', 'team2']).apply(lambda x: (x['winner'] == x['team1']).sum()).reset_index(name='h2h_team1_wins')

# Merge calculated wins back to the original DataFrame
summary = summary.merge(h2h_wins, on=['team1', 'team2'], how='left')
summary["h2h_team1_wins%"] = round((summary["h2h_team1_wins"] / summary["h2h"]) * 100).fillna(0).astype(int)

# Display the updated DataFrame
summary[['match_id', 'team1', 'team2', 'h2h', 'h2h_team1_wins']]

C:\Users\harik\AppData\Local\Temp\ipykernel_12220\3526048767.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  h2h_wins = summary.groupby(['team1', 'team2']).apply(lambda x: (x['winner'] == x['team1']).sum()).reset_index(name='h2h_team1_wins')


,match_id,team1,team2,h2h,h2h_team1_wins
0,1463667,United Arab Emirates,Kuwait,7,6
1,1463666,Oman,Kuwait,4,3
2,1463665,Qatar,Bahrain,6,4
3,1463664,United Arab Emirates,Saudi Arabia,4,3
4,1463663,United Arab Emirates,Qatar,6,6
...,...,...,...,...,...
8519,225271,Sri Lanka,England,14,4
8520,225263,Pakistan,England,30,9
8521,222678,South Africa,New Zealand,15,11
8522,211048,New Zealand,Australia,19,6


### **Step 5: Head-to-Head Winning Count at a Venue**

In [6]:
# Standardize text data
summary['team1'] = summary['team1'].str.strip().str.title()
summary['team2'] = summary['team2'].str.strip().str.title()
summary['ground'] = summary['ground'].str.strip().str.title().fillna('Unknown')
summary['winner'] = summary['winner'].str.strip().str.title()

# Calculate matches played and wins at the same venue for team1
summary['h2h_venue'] = summary.groupby(['team1', 'team2', 'ground'])['match_id'].transform('count')
head_to_head = summary.groupby(['team1', 'team2', 'ground', 'winner']).size().unstack(fill_value=0)

# Create a column for wins by team1 at the same venue
def get_team1_wins(row):
    try:
        return head_to_head.loc[(row['team1'], row['team2'], row['ground']), row['team1']]
    except KeyError:
        return 0

summary['h2h_venue_team1_wins'] = summary.apply(get_team1_wins, axis=1)
summary['h2h_venue_team1_wins%'] = round((summary['h2h_venue_team1_wins'] / summary['h2h_venue']) * 100, 2).fillna(0).astype(int)

# Display the updated DataFrame
summary[['match_id', 'match_date', 'team1', 'team2', 'ground', 'winner', 'h2h_venue', 'h2h_venue_team1_wins', 'h2h_venue_team1_wins%']].head()


,match_id,match_date,team1,team2,ground,winner,h2h_venue,h2h_venue_team1_wins,h2h_venue_team1_wins%
0,1463667,2024-12-21,United Arab Emirates,Kuwait,"Dubai International Cricket Stadium, Dubai",United Arab Emirates,1,1,100
1,1463666,2024-12-20,Oman,Kuwait,"Icc Global Cricket Academy, Dubai",Kuwait,1,0,0
2,1463665,2024-12-20,Qatar,Bahrain,"Icc Global Cricket Academy, Dubai",Qatar,1,1,100
3,1463664,2024-12-19,United Arab Emirates,Saudi Arabia,"Icc Global Cricket Academy, Dubai",Saudi Arabia,1,0,0
4,1463663,2024-12-18,United Arab Emirates,Qatar,"Icc Global Cricket Academy, Dubai",United Arab Emirates,1,1,100


In [7]:
norway = summary[((summary["team1"] == "Norway") & (summary["team2"] == "Finland") | (summary["team1"] == "Finland") & (summary["team2"] == "Norway")) 
                 & (summary["ground"] == "Kerava National Cricket Ground, Kerava")].reset_index(drop=True)
norway

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,h2h,h2h_team1_wins,h2h_team1_wins%,h2h_venue,h2h_venue_team1_wins,h2h_venue_team1_wins%
0,1438262,2024,2024-06-16,Nordic T20 Cup,Norway,Finland,Finland,bat,Norway,"Kerava National Cricket Ground, Kerava",4,4,100,3,3,100
1,1438259,2024,2024-06-15,Nordic T20 Cup,Norway,Finland,Finland,bat,Norway,"Kerava National Cricket Ground, Kerava",4,4,100,3,3,100
2,1438257,2024,2024-06-14,Nordic T20 Cup,Norway,Finland,Finland,field,Norway,"Kerava National Cricket Ground, Kerava",4,4,100,3,3,100


### **Step 6: Team Wins at a Venue**

In [8]:
import pandas as pd

# Assuming 'summary' is your DataFrame
# And 'head_to_head' is a DataFrame with team wins data

# Calculate matches played at the same venue for team1 and team2
summary['team1_venue'] = summary.groupby(['team1', 'ground'])['match_id'].transform('count')
summary['team2_venue'] = summary.groupby(['team2', 'ground'])['match_id'].transform('count')

# Function to get team wins
def get_team_wins(row, team_col):
    try:
        return head_to_head.loc[(row['team1'], row['team2'], row['ground']), row[team_col]]
    except KeyError:
        return 0
    except:
        return 0

# Apply the function to calculate wins at the venue
summary['team1_venue_wins'] = summary.apply(lambda row: get_team_wins(row, 'team1'), axis=1)
summary['team2_venue_wins'] = summary.apply(lambda row: get_team_wins(row, 'team2'), axis=1)

# Calculate win percentages
summary['team1_venue_wins%'] = round((summary['team1_venue_wins'] / summary['team1_venue']) * 100, 2).fillna(0).astype(int)
summary['team2_venue_wins%'] = round((summary['team2_venue_wins'] / summary['team2_venue']) * 100, 2).fillna(0).astype(int)

# Display the updated DataFrame
display_cols = ['match_id', 'match_date', 'team1', 'team2', 'team1_venue_wins', 'team1_venue_wins%', 'team2_venue_wins', 'team2_venue_wins%', 'ground', 'winner']
summary[display_cols].head()


,match_id,match_date,team1,team2,team1_venue_wins,team1_venue_wins%,team2_venue_wins,team2_venue_wins%,ground,winner
0,1463667,2024-12-21,United Arab Emirates,Kuwait,1,8,0,0,"Dubai International Cricket Stadium, Dubai",United Arab Emirates
1,1463666,2024-12-20,Oman,Kuwait,0,0,1,25,"Icc Global Cricket Academy, Dubai",Kuwait
2,1463665,2024-12-20,Qatar,Bahrain,1,50,0,0,"Icc Global Cricket Academy, Dubai",Qatar
3,1463664,2024-12-19,United Arab Emirates,Saudi Arabia,0,0,1,100,"Icc Global Cricket Academy, Dubai",Saudi Arabia
4,1463663,2024-12-18,United Arab Emirates,Qatar,1,7,0,0,"Icc Global Cricket Academy, Dubai",United Arab Emirates


### **Step 7: Last 5 Matches Performance**

In [9]:
import pandas as pd

# Function to calculate last 5 matches wins for each match
def calculate_last_5_wins(df):
    df['team1_last_5_wins'] = 0
    df['team2_last_5_wins'] = 0
    
    teams = set(df['team1']).union(set(df['team2']))
    for team in teams:
        team_matches = df[(df['team1'] == team) | (df['team2'] == team)].sort_values(by=['match_date', 'match_id'], ascending=False)
        wins_count = []
        for i in range(len(team_matches)):
            match = team_matches.iloc[i]
            prev_matches = team_matches.iloc[i+1:i+6]  # next 5 matches in descending order
            wins = len(prev_matches[prev_matches['winner'] == team])
            wins_count.append(wins)
        
        # Ensure wins_count is aligned with the correct matches in the main DataFrame
        df.loc[df[df['team1'] == team].index, 'team1_last_5_wins'] = wins_count[:len(df[df['team1'] == team])]
        df.loc[df[df['team2'] == team].index, 'team2_last_5_wins'] = wins_count[:len(df[df['team2'] == team])]

    return df

# Calculate last 5 matches wins for each match
summary = calculate_last_5_wins(summary)

# Display the DataFrame
summary[['match_id', 'team1', 'team2', 'match_date', 'team1_last_5_wins', 'team2_last_5_wins']][:5]


,match_id,team1,team2,match_date,team1_last_5_wins,team2_last_5_wins
0,1463667,United Arab Emirates,Kuwait,2024-12-21,4,3
1,1463666,Oman,Kuwait,2024-12-20,2,2
2,1463665,Qatar,Bahrain,2024-12-20,1,2
3,1463664,United Arab Emirates,Saudi Arabia,2024-12-19,5,2
4,1463663,United Arab Emirates,Qatar,2024-12-18,5,1


### **Step 8: Event Name Standardization**

In [10]:
# Standardize event column based on team participation
team_event_counts = summary.groupby(['team1', 'team2'])['event_name'].nunique().reset_index()
multiple_events_teams = team_event_counts[team_event_counts['event_name'] > 1]
teams_in_multiple_events = set(multiple_events_teams['team1']).union(set(multiple_events_teams['team2']))

# List of events to be considered as T20I
t20i_events = [
    'T20I',
    'Central American Cricket Championships',
    'South American Men\'s Championships'
]

# Apply the condition to set the 'event' column
if match_type == "T20":
    summary['event'] = summary.apply(lambda row: 'T20I' if row['event_name'] in t20i_events or row['team1'] in teams_in_multiple_events or row['team2'] 
                                     in teams_in_multiple_events else row['event_name'], axis=1)

# Check if the match type is "ODI"
elif match_type == "ODI":
    summary['event'] = summary.apply(lambda row: 'ODI' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Check if the match type is "ODM"
elif match_type == "ODM":
    summary['event'] = summary.apply(lambda row: 'ODM' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Check if the match type is "MDM"
elif match_type == "MDM":
    summary['event'] = summary.apply(lambda row: 'MDM' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# If none of the above match types, assume it is a Test match
elif match_type == "Test":
    summary['event'] = summary.apply(lambda row: 'Test' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Display the first 10 rows of the summary DataFrame to check the updates
summary[:5]


,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,event
0,1463667,2024,2024-12-21,Gulf T20I Championship,United Arab Emirates,Kuwait,United Arab Emirates,bat,United Arab Emirates,"Dubai International Cricket Stadium, Dubai",...,100,12,1,1,0,8,0,4,3,T20I
1,1463666,2024,2024-12-20,Gulf T20I Championship,Oman,Kuwait,Oman,bat,Kuwait,"Icc Global Cricket Academy, Dubai",...,0,3,4,0,1,0,25,2,2,T20I
2,1463665,2024,2024-12-20,Gulf T20I Championship,Qatar,Bahrain,Qatar,field,Qatar,"Icc Global Cricket Academy, Dubai",...,100,2,5,1,0,50,0,1,2,T20I
3,1463664,2024,2024-12-19,Gulf T20I Championship,United Arab Emirates,Saudi Arabia,United Arab Emirates,field,Saudi Arabia,"Icc Global Cricket Academy, Dubai",...,0,13,1,0,1,0,100,5,2,T20I
4,1463663,2024,2024-12-18,Gulf T20I Championship,United Arab Emirates,Qatar,United Arab Emirates,bat,United Arab Emirates,"Icc Global Cricket Academy, Dubai",...,100,13,3,1,0,7,0,5,1,T20I


In [11]:
summary['event'].unique()

array(['T20I', 'Nepal Premier League', 'Bangladesh Premier League',
       'Super Smash',
       "ICC Men's T20 World Cup Sub Regional Africa Qualifier", 'SA20',
       'Syed Mushtaq Ali Trophy', 'CSA T20 Challenge', 'Big Bash League',
       'Major League Cricket',
       'Cricket Ireland Inter-Provincial Twenty20 Trophy',
       'Caribbean Premier League', 'Lanka Premier League', 'IPL',
       'Vitality Blast', "The Hundred Men's Competition",
       'Pakistan Super League', 'International League T20',
       'Mzansi Super League'], dtype=object)

In [12]:
summary[summary['event'] == "Big Bash League"]

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,event
370,1443087,2025,2025-01-11,Big Bash League,Brisbane Heat,Adelaide Strikers,Brisbane Heat,field,Adelaide Strikers,"Adelaide Oval, Adelaide",...,60,10,74,6,4,60,5,2,1,Big Bash League
371,1443086,2025,2025-01-11,Big Bash League,Sydney Sixers,Perth Scorchers,Perth Scorchers,field,Sydney Sixers,"Sydney Cricket Ground, Sydney",...,58,51,12,7,5,13,41,3,2,Big Bash League
372,1443085,2025,2025-01-10,Big Bash League,Sydney Thunder,Hobart Hurricanes,Hobart Hurricanes,field,Hobart Hurricanes,"Bellerive Oval, Hobart",...,20,10,44,2,8,20,18,3,4,Big Bash League
373,1443084,2025,2025-01-09,Big Bash League,Sydney Sixers,Melbourne Stars,Sydney Sixers,field,Melbourne Stars,"Melbourne Cricket Ground, Melbourne",...,63,12,27,7,4,58,14,4,2,Big Bash League
374,1443082,2025,2025-01-07,Big Bash League,Perth Scorchers,Melbourne Renegades,Melbourne Renegades,field,Melbourne Renegades,"Perth Stadium, Perth",...,66,28,6,4,2,14,33,2,2,Big Bash League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,524919,2011,2011-12-20,Big Bash League,Melbourne Stars,Brisbane Heat,Melbourne Stars,bat,Melbourne Stars,"Brisbane Cricket Ground, Woolloongabba, Brisbane",...,44,9,51,4,5,44,9,3,2,Big Bash League
8085,524918,2011,2011-12-18,Big Bash League,Perth Scorchers,Hobart Hurricanes,Hobart Hurricanes,bat,Hobart Hurricanes,"Western Australia Cricket Association Ground, ...",...,66,23,3,2,1,8,33,3,1,Big Bash League
8086,524917,2011,2011-12-18,Big Bash League,Melbourne Renegades,Adelaide Strikers,Melbourne Renegades,field,Adelaide Strikers,"Adelaide Oval, Adelaide",...,40,10,74,4,6,40,8,1,0,Big Bash League
8087,524916,2011,2011-12-17,Big Bash League,Sydney Thunder,Melbourne Stars,Sydney Thunder,field,Sydney Thunder,"Melbourne Cricket Ground, Melbourne",...,50,11,27,4,4,36,14,0,5,Big Bash League


### **Step 9: Capitalizing IPL teams**

In [13]:
# Create a mask for rows where event_name is "IPL"
mask = summary["event_name"] == "IPL"

# Convert the specified columns to uppercase only for the rows where event_name is "IPL"
summary.loc[mask, 'team1'] = summary.loc[mask, 'team1'].str.upper()
summary.loc[mask, 'team2'] = summary.loc[mask, 'team2'].str.upper()
summary.loc[mask, 'toss_winner'] = summary.loc[mask, 'toss_winner'].str.upper()
summary.loc[mask, 'winner'] = summary.loc[mask, 'winner'].str.upper()

### **Step 10: Save Cleaned DataFrame**

In [14]:
# Select and reorder final columns
summary_df = summary[['match_id', 'season', 'event_name', 'event', 'match_date', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'h2h_team1_wins%', 'h2h_venue_team1_wins%', 'team1_venue_wins%', 'team2_venue_wins%', 'team1_last_5_wins', 'team2_last_5_wins', 'ground']].copy()

# Save the cleaned summary DataFrame to CSV
summary_df.to_csv(fr"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\{gender}\{match_type}_{gender}_match_summary_cleaned.csv", index=False)

summary_df.head()

,match_id,season,event_name,event,match_date,team1,team2,toss_winner,toss_decision,winner,h2h_team1_wins%,h2h_venue_team1_wins%,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,ground
0,1463667,2024,Gulf T20I Championship,T20I,2024-12-21,United Arab Emirates,Kuwait,United Arab Emirates,bat,United Arab Emirates,86,100,8,0,4,3,"Dubai International Cricket Stadium, Dubai"
1,1463666,2024,Gulf T20I Championship,T20I,2024-12-20,Oman,Kuwait,Oman,bat,Kuwait,75,0,0,25,2,2,"Icc Global Cricket Academy, Dubai"
2,1463665,2024,Gulf T20I Championship,T20I,2024-12-20,Qatar,Bahrain,Qatar,field,Qatar,67,100,50,0,1,2,"Icc Global Cricket Academy, Dubai"
3,1463664,2024,Gulf T20I Championship,T20I,2024-12-19,United Arab Emirates,Saudi Arabia,United Arab Emirates,field,Saudi Arabia,75,0,0,100,5,2,"Icc Global Cricket Academy, Dubai"
4,1463663,2024,Gulf T20I Championship,T20I,2024-12-18,United Arab Emirates,Qatar,United Arab Emirates,bat,United Arab Emirates,100,100,7,0,5,1,"Icc Global Cricket Academy, Dubai"
